In [ ]:
!pip install nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/AI ML Bootcamp/song.csv')
df.head()

,name,collection,genre,content
0,আনুষ্ঠানিক,আনুষ্ঠানিক,গান,দুইটি হৃদয়ে একটি আসন পাতিয়া বসো হে হৃদয়নাথ।\nক...
1,ভানুসিংহ ঠাকুরের পদাবলী,ভানুসিংহ ঠাকুরের পদাবলী,গান,"বসন্ত আওল রে!\nমধুকর গুন গুন,\nঅমুয়ামঞ্জরী কান..."
2,শ্যামা,গীতিনাট্য এবং নৃত্যনাট্য,গান,প্রথম দৃশ্য\nবজ্রসেন ও তাহার বন্ধু\nবন্ধু। তুম...
3,চিত্রাঙ্গদা,গীতিনাট্য এবং নৃত্যনাট্য,গান,ভূমিকা\nপ্রভাতের আদিম অভাস অরুণবর্ণ আভার আবরণে...
4,বাল্মীকিপ্রতিভা,গীতিনাট্য এবং নৃত্যনাট্য,গান,"প্রথম দৃশ্য\nঅরণ্য\nবনদেবীগণ\nসহে\nনা, সহে না,..."


**Tokenizing the text**

In [ ]:
import nltk
# Downloading 'punkt_tab' which is required for this example
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

# Tokenizing the content of the dataset
tokenized_texts = [word_tokenize(text) for text in df['content']]

# Flatten the list of tokens and create a vocabulary
flattened_tokens = [token for sublist in tokenized_texts for token in sublist]
vocabulary = list(set(flattened_tokens))

print("Vocabulary Size:", len(vocabulary))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Vocabulary Size: 23325


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Constants
SEQ_LENGTH = 20  # Length of input sequences
VOCAB_SIZE = 23325  # Vocabulary size
EMBEDDING_DIM = 128  # Embedding dimension
BATCH_SIZE = 64


**Building GAN model by generatng Generator and Discriminator**

In [ ]:
# Define the Generator
def create_generator(seq_length, vocab_size, embedding_dim):
    model = tf.keras.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        layers.LSTM(128, return_sequences=True),  # (batch_size, seq_length, 128)
        layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))  # (batch_size, seq_length, vocab_size)
    ])
    return model

# Define the Discriminator
def create_discriminator(seq_length, vocab_size, embedding_dim):
    model = tf.keras.Sequential([
        layers.Input(shape=(seq_length, vocab_size)),  # (batch_size, seq_length, vocab_size)
        layers.LSTM(128, return_sequences=False),  # (batch_size, 128)
        layers.Dense(1, activation='sigmoid')  # Output: (batch_size, 1) binary classification
    ])
    return model

# Create the GAN Model
def create_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator during GAN training

    noise_input = tf.keras.Input(shape=(SEQ_LENGTH,))  # Input shape: (batch_size, seq_length)
    generated_sequence = generator(noise_input)  # Output shape: (batch_size, seq_length, vocab_size)
    validity = discriminator(generated_sequence)  # Output shape: (batch_size, 1)

    gan = tf.keras.Model(noise_input, validity)  # Define the full GAN model
    gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')
    return gan


**Defining parameters**

In [ ]:
# Define sequence length, vocab size, and embedding dimension
SEQ_LENGTH = 20  # for example
VOCAB_SIZE = 5000  # example vocabulary size
EMBEDDING_DIM = 100  # example embedding dimension

# Create the Generator and Discriminator
generator = create_generator(SEQ_LENGTH, VOCAB_SIZE, EMBEDDING_DIM)
discriminator = create_discriminator(SEQ_LENGTH, VOCAB_SIZE, EMBEDDING_DIM)

# Create the GAN model
gan = create_gan(generator, discriminator)
gan.summary()  # Optional: Check the model summary to confirm it's correct


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)          │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_6 (Sequential)            │ (None, 20, 5000)            │       1,262,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_7 (Sequential)            │ (None, 1)                   │       2,626,177 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,888,425 (14.83 MB)

 Trainable params: 1,262,248 (4.82 MB)

 Non-trainable params: 2,626,177 (10.02 MB)

**Training the GAN Model**

In [ ]:
# Define the train_gan function
def train_gan(generator, discriminator, gan, real_data, epochs=1000, batch_size=64):
    half_batch = batch_size // 2

In [ ]:
# Define and compile the generator model
generator = create_generator(SEQ_LENGTH, VOCAB_SIZE, EMBEDDING_DIM)
generator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Define and compile the discriminator model
discriminator = create_discriminator(SEQ_LENGTH, VOCAB_SIZE, EMBEDDING_DIM)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Create the GAN model
gan = create_gan(generator, discriminator)

# Compile the GAN model (this is needed because you’re training the GAN as a whole)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))


In [ ]:
# Assuming you have `real_data` prepared for training
train_gan(generator, discriminator, gan, real_data, epochs=1000, batch_size=64)


**Generating synthetic Text**

In [ ]:
def generate_synthetic_sequences(generator, num_sequences=10):
    noise = np.random.randint(0, VOCAB_SIZE, (num_sequences, SEQ_LENGTH))
    generated_sequences_prob = generator.predict(noise)
    generated_sequences = np.argmax(generated_sequences_prob, axis=-1)  # Convert to token IDs
    return generated_sequences

# Generate synthetic sequences
synthetic_sequences = generate_synthetic_sequences(generator, num_sequences=5)
print("Generated Synthetic Sequences (Token IDs):", synthetic_sequences)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
Generated Synthetic Sequences (Token IDs): [[14782 14737 10224  6039  1318   101  4674  9983  2957 10303 18466  4114
  11642 11271   948 15803 15803 21737 19431 11451]
 [ 6348 20361  2622  2622  2622  3025 12641 12641 12641 11479  7364  3540
   7024  7024 14304 21564 20830 15259  7777 12062]
 [ 5956 11417 18639  2585   175 20594 20594 16109  9096  8488    77 22734
  22393 10503 22393 22393  4186  4186  4186    21]
 [ 9513  9513  1554  2957 20265 10000 16044 16882 10822 10822  5232  7792
  20584 11475 12652 11417  6691  3262 12827  2416]
 [ 3537  3537 17593 17593 22660  1836  5835 18975 18497  4174  4174 17123
   6447  1304 10754  7400 10187 23052 23324  1476]]


In [ ]:
def generate_text(generator, vocab_size, seq_length, num_samples=10):
    random_noise = np.random.randint(0, vocab_size, (num_samples, seq_length))
    generated_sequences = generator.predict(random_noise)

    # Convert token IDs back to words
    for seq in generated_sequences:
        words = [vocabulary[np.argmax(word_probs)] for word_probs in seq]
        print(' '.join(words))

# Generate sample texts
generate_text(generator, VOCAB_SIZE, SEQ_LENGTH)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
চাহিছে। আয়– টানি ঘনায়ে বুক। কালীকে। ‍সে অসহ– পাইয়াছ খেলারই করিবে ভাই। গেঁথেছিলেম ঝরনাতলায় কম্পিছে সুরনর অবসাদে। দেহদুর্গে মায়া-তান পাবনা
নেহারো হদয়কোণে॥ দ্বিতীয়া হদয়কোণে॥ ষে। পরিচয়॥ রাগরাগিণীতে পরিচয়॥ পাখিটি সঞ্চয়। মাঝ-কিনারায় নাগিনী। বিনি জাগিয়াছে হলে দাঁড়ালেন মিলনদিনের বদ্ধ জ্বলবে॥ প
আসিল জল– জল– জল– খেলায়– খুঁজি॥ শিউলিফুলে॥ বোলব বাজাইছে ধরিনু বাণে তোমা-পানে– শিরে দুলিল পাশরে– মুখে মাটি-যে সোহাগিনির চিহ্ন ডালা–
ভিন্নছন্দ চোরাবালির যত্নে ক্ষমহ কাড়ব খেয়াতরীর কিছুতে কপোতকূজনকরুণ গেঁথেছি মহাপ্রাণ বাহিরিব। পানে। দীক্ষা॥ কলি কলি কাড়িয়া প্রহরী॥ সুকঠিন চিতচাতক চিন্তিত
নবঘনমন্দ্রে॥ ছায়ারে বরষাজলধারার অরূপ-রসের ঝিল্লিরবে ঝিল্লিরবে দিগন্তরে। ঢাকা। মূর্ছিত সুখ- টানল। কুঞ্জবিতানে কুঞ্জবিতানে চেতনায় আনবে বধিল বিশ্ববিধাতার দুঃখতাপ দুঃখতাপ গীতি॥
কাঁপছে আ জয়ধ্বজা না-জানার হরষ চরণসেবার শশী ওঠেন মুরদখানা অনিল হোক-না নেই– ছলে॥ আনিবে আনিবে সুগন্ধিত আধো-আধো পুরোনো বজ্রশিখায় বনগন্ধে॥
বসো তারকমালিনী নিবুক রসধারা। সভয়-তবধ ধরেছে ভৃঙ্গ-সম দিবারাতি॥ দিবারাতি॥ হৃদয়বনচ্ছায়ে র